<a href="https://colab.research.google.com/github/parvezmmr/AdBuss/blob/main/delhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import pandas as pd
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
from folium import * # plotting library
from bs4 import BeautifulSoup
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


**getting requred dataframe of List of neighbourhoods in Pune. scraping web page**

In [ ]:


req = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Delhi").text
soup = BeautifulSoup(req, 'lxml')
div = soup.find('div', class_="mw-parser-output")
print("web Page Imported")
#Code to extract the relevent data from the request object using beautiful soup
data = pd.DataFrame(columns=['Borough','Neighborhood'])
i=-1
flag = False
no=0
prev_borough = None
for child in div.children:
    if child.name:
        span = child.find('span')
        if span!=-1 and span is not None:
            try:
                if span['class'][0] == 'mw-headline' and child.a.text!='edit':
                    prev_borough = child.a.text
                    i+=1
                    flag = True
                    continue
            except KeyError:
                continue
        if child.name=='ul' and flag==True:
            neighborhood = []
            for ch in child.children:
                
                try:
                    data.loc[no]=[prev_borough,ch.text]
                    no+=1
                except AttributeError:
                    pass
        flag = False
data[50:60]
dfn= pd.DataFrame(data)
dfn.shape

web Page Imported


(114, 2)

In [ ]:
dfn.to_csv('/content/delhi_csv.csv', index=False)

Now,We Getting Location Data From Geolocator by providing List_of_neighbourhoods_in_Mumbai data



In [ ]:
lat_lng = pd.DataFrame(columns=['latitude','longitude'])
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(114):
    address = data['Neighborhood'].loc[i]+',Delhi'
    try: 
        location = geolocator.geocode(address)
        lat_lng.loc[i]=[location.latitude,location.longitude]
    except AttributeError:
        continue
df1 = data
df2 = lat_lng
delhi_neighbourhood_data = pd.concat([df1, df2], axis=1)

In [ ]:
delhi_neighbourhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614193,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Begum Pur,28.725503,77.058371
3,North West Delhi,Karala,28.735140,77.032511
4,North West Delhi,Model Town,28.702714,77.193991


In [ ]:
delhi_neighbourhood_data.shape


(114, 4)

In [ ]:
delhi_neighbourhood_data.to_csv('/content/delhi_lag_lat_csv.csv', index=False)


In [ ]:

df1=pd.read_csv('/content/delhi_csv.csv')
df1.shape

(114, 2)

In [ ]:
df1.head()


,Borough,Neighborhood
0,North West Delhi,Adarsh Nagar
1,North West Delhi,Ashok Vihar
2,North West Delhi,Begum Pur
3,North West Delhi,Karala
4,North West Delhi,Model Town


# **Analysis of neighbourhoods using map**
Creating Folium Map of List of neighbourhoods in Delhi

In [ ]:

import folium
latitude = df2.loc[0, 'latitude']
longitude = df2.loc[0, 'longitude']
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map

for lat, lng, label in zip(df2['latitude'],df2['longitude'],df1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi 



Getting Data Using Foursquare API by providing ID and Secret.*italicized text*

In [ ]:

CLIENT_ID = 'MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR' # your Foursquare ID
CLIENT_SECRET = '10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR
CLIENT_SECRET:10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA


In [ ]:
latitude = df2.loc[0, 'latitude']
longitude = df2.loc[0, 'longitude']
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [ ]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize

In [ ]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Bikanerwala,Indian Restaurant,28.613391,77.076084
1,Uttam nagar,Metro Station,28.620201,77.068709
2,Gold's Gym A Block Janakpuri,Gym,28.622439,77.069348
3,Potholes at Dabri,Pool,28.605309,77.072504


In [ ]:

nearby_venues.shape

(4, 4)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
  

In [ ]:
delhi_venues = getNearbyVenues(names =df1 ['Neighborhood'],
                               latitudes =df2 ['latitude'],
                               longitudes =df2 ['longitude'])

Adarsh Nagar
Ashok Vihar
Begum Pur
Karala
Model Town
Narela
Pitam Pura
Rohini Sub City
Shalimar Bagh
Civil Lines
Gulabi Bagh
Kamla Nagar
Kashmiri Gate
Kotwali
Mori Gate; Delhi
Sadar Bazaar
Sarai Rohilla
Shakti Nagar
Tis Hazari
Timarpur
Wazirabad
Dilshad Garden
Naveen Shahdara
New Usmanpur
Shahdara
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Shastri Nagar
Kishanganj
Paharganj
Rajender Nagar
Barakhamba Road
Chanakyapuri
Connaught Place
East Vinod Nagar
Jhilmil Colony
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Preet Vihar
Anand Vihar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Alaknanda
Chattarpur
Chittaranjan Park
Defence Colony
East of Kailash
Govindpuri
Greater Kailash
Green Park
Gulmohar Park
Hauz Khas
Hauz Khas Village
Jangpura
Jor Bagh
Kalkaji
Khanpur
Lajpat Nagar
Lodi Colony
Malviya Nagar
Mehrauli
Neeti Bagh
Nehru Place
Netaji Nagar
New Friends Colony
Nizamuddin West
Okhla
Safdarjung Enclave
Sainik Farm
Saket
Sangam Vihar
Sarita Vihar
Sarojini Nagar
Sarvoday

In [ ]:
print(delhi_venues.shape)
delhi_venues.head()

(2168, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Adarsh Nagar,28.614193,77.071541,Uttam nagar,28.620201,77.068709,Metro Station
2,Adarsh Nagar,28.614193,77.071541,Gold's Gym A Block Janakpuri,28.622439,77.069348,Gym
3,Adarsh Nagar,28.614193,77.071541,Potholes at Dabri,28.605309,77.072504,Pool
4,Ashok Vihar,28.699453,77.184826,Bellagio,28.696361,77.180021,Asian Restaurant


In [ ]:
import pandas as pd

delhi_venues.to_csv('/content/delhi_venues.csv', index=False)

In [ ]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Nagar,4,4,4,4,4,4
Alaknanda,52,52,52,52,52,52
Anand Vihar,6,6,6,6,6,6
Ashok Nagar,16,16,16,16,16,16
Ashok Vihar,9,9,9,9,9,9
...,...,...,...,...,...,...
Vasundhara Enclave,10,10,10,10,10,10
Vishwas Nagar,26,26,26,26,26,26
Vivek Vihar,9,9,9,9,9,9


In [ ]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

There are 190 uniques categories.


# Creating Onehot encoding

In [ ]:
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Botanical Garden,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,...,Punjabi Restaurant,Racetrack,Record Shop,Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
delhi_onehot.shape

(2168, 190)

In [ ]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Botanical Garden,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,...,Punjabi Restaurant,Racetrack,Record Shop,Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,Adarsh Nagar,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,Alaknanda,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057692,0.0,0.0,0.000000,0.0,0.038462,0.019231,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057692,0.0,0.038462,0.0,0.0,0.076923,...,0.0,0.0,0.0,0.000000,0.057692,0.0,0.0,0.057692,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019231,0.0,0.019231,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.0625,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.062500,...,0.0,0.0,0.0,0.000000,0.062500,0.0,0.0,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.000000,0.0,0.111111,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.100000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.100000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.1,0.

# getting Freq of TOP 5 venues.

In [ ]:
num_top_venues = 5

for hood in delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = delhi_grouped[delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Nagar----
               venue  freq
0  Indian Restaurant  0.25
1               Pool  0.25
2                Gym  0.25
3      Metro Station  0.25
4      Women's Store  0.00


----Alaknanda----
               venue  freq
0  Indian Restaurant  0.12
1        Coffee Shop  0.08
2               Café  0.06
3        Pizza Place  0.06
4     Sandwich Place  0.06


----Anand Vihar----
           venue  freq
0            Gym  0.33
1    Pizza Place  0.17
2  Shopping Mall  0.17
3  Big Box Store  0.17
4          Hotel  0.17


----Ashok Nagar----
                            venue  freq
0                           Hotel  0.12
1               Indian Restaurant  0.12
2            Fast Food Restaurant  0.06
3                          Garden  0.06
4  Multicuisine Indian Restaurant  0.06


----Ashok Vihar----
                  venue  freq
0           Pizza Place  0.11
1      Asian Restaurant  0.11
2  Fast Food Restaurant  0.11
3           Coffee Shop  0.11
4        Sandwich Place  0.11


----Badar

# Getting top 10 venues of all neighbourhoods.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Pool,Metro Station,Gym,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,Alaknanda,Indian Restaurant,Coffee Shop,Café,Restaurant,Market,Dessert Shop,Pizza Place,Asian Restaurant,Sandwich Place,Hotel
2,Anand Vihar,Gym,Shopping Mall,Pizza Place,Big Box Store,Hotel,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
3,Ashok Nagar,Indian Restaurant,Hotel,Market,Pizza Place,Coffee Shop,Multicuisine Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Food Truck
4,Ashok Vihar,Indian Restaurant,Asian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Fast Food Restaurant,Sandwich Place,South Indian Restaurant,Snack Place,Hookah Bar
...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,Metro Station,Juice Bar,BBQ Joint,Athletics & Sports,Asian Restaurant,Track,Pool,Shop & Service,Diner,Hotel
104,Vishwas Nagar,Coffee Shop,BBQ Joint,Indian Restaurant,Restaurant,Market,Asian Restaurant,Sandwich Place,Dessert Shop,Café,Mediterranean Restaurant
105,Vivek Vihar,Indian Restaurant,Café,Farm,Resort,Food Truck,Coffee Shop,Restaurant,French Restaurant,Food Court,Food & Drink Shop
106,Wazirabad,Indian Restaurant,Shop & Service,Pizza Place,Diner,Metro Station,Light Rail Station,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck


# Lets Do clustering For neighbourhoods and venue 5 Cluster

In [ ]:
from sklearn.cluster import KMeans

kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 0, 2, 2, 3, 3, 3, 4, 3], dtype=int32)

In [ ]:
neighborhoods_venues_sorted['Neighborhood']


0            Adarsh Nagar
1               Alaknanda
2             Anand Vihar
3             Ashok Nagar
4             Ashok Vihar
              ...        
103    Vasundhara Enclave
104         Vishwas Nagar
105           Vivek Vihar
106             Wazirabad
107          Yamuna Vihar
Name: Neighborhood, Length: 108, dtype: object

In [ ]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Pool,Metro Station,Gym,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop
1,Alaknanda,Indian Restaurant,Coffee Shop,Café,Restaurant,Market,Dessert Shop,Pizza Place,Asian Restaurant,Sandwich Place,Hotel
2,Anand Vihar,Gym,Shopping Mall,Pizza Place,Big Box Store,Hotel,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
3,Ashok Nagar,Indian Restaurant,Hotel,Market,Pizza Place,Coffee Shop,Multicuisine Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Food Truck
4,Ashok Vihar,Indian Restaurant,Asian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Fast Food Restaurant,Sandwich Place,South Indian Restaurant,Snack Place,Hookah Bar


In [ ]:
delhi_merged = neighborhoods_venues_sorted

# add clustering labels
delhi_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

delhi_merged.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Adarsh Nagar,Indian Restaurant,Pool,Metro Station,Gym,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,3
1,Alaknanda,Indian Restaurant,Coffee Shop,Café,Restaurant,Market,Dessert Shop,Pizza Place,Asian Restaurant,Sandwich Place,Hotel,3
2,Anand Vihar,Gym,Shopping Mall,Pizza Place,Big Box Store,Hotel,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,0
3,Ashok Nagar,Indian Restaurant,Hotel,Market,Pizza Place,Coffee Shop,Multicuisine Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Food Truck,2
4,Ashok Vihar,Indian Restaurant,Asian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Fast Food Restaurant,Sandwich Place,South Indian Restaurant,Snack Place,Hookah Bar,2


# Appending Lag Lat dataframe to above dataset

In [ ]:
lag_lat=pd.read_csv("//content/delhi_csv.csv")
dflag=df2['latitude']
dflat=df2['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
delhi_sorted=pd.concat([neighborhoods_venues_sorted, lag_lat], axis=1)
delhi_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,latitude,longitude
0,Adarsh Nagar,Indian Restaurant,Pool,Metro Station,Gym,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,3.0,28.614193,77.071541
1,Alaknanda,Indian Restaurant,Coffee Shop,Café,Restaurant,Market,Dessert Shop,Pizza Place,Asian Restaurant,Sandwich Place,Hotel,3.0,28.699453,77.184826
2,Anand Vihar,Gym,Shopping Mall,Pizza Place,Big Box Store,Hotel,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,0.0,28.725503,77.058371
3,Ashok Nagar,Indian Restaurant,Hotel,Market,Pizza Place,Coffee Shop,Multicuisine Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Food Truck,2.0,28.735140,77.032511
4,Ashok Vihar,Indian Restaurant,Asian Restaurant,Donut Shop,Pizza Place,Coffee Shop,Fast Food Restaurant,Sandwich Place,South Indian Restaurant,Snack Place,Hookah Bar,2.0,28.702714,77.193991


In [ ]:
delhi_sorted.shape

(114, 14)


# DATA Analysis start

# Creating Folium Map of Cluster we just define.

In [ ]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = delhi_sorted.loc[0, 'latitude']
longitude = delhi_sorted.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, poi,cluster in zip(delhi_sorted['latitude'],delhi_sorted['longitude'],delhi_sorted['Neighborhood'],delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters

TypeError: ignored

# Show Data of common venue from each cluster

## *first* *cluster* *i*.*e* '*cluster* *0*'

In [ ]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Gym,Hotel,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,0
14,Hotel,Train Station,Juice Bar,Asian Restaurant,Food Court,Food,Food & Drink Shop,0
18,Light Rail Station,Farm,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,0
20,Light Rail Station,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,0
22,Shopping Mall,Wings Joint,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,0
32,Mobile Phone Shop,Fast Food Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,0
39,Light Rail Station,Farm,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,0
46,Shopping Mall,Movie Theater,Moving Target,Light Rail Station,Department Store,Clothing Store,Train Station,0
50,Dessert Shop,BBQ Joint,Fast Food Restaurant,South Indian Restaurant,Pizza Place,Mediterranean Restaurant,Arcade,0
65,Light Rail Station,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,0


## *second* *cluster* *i*.*e* '*cluster* *1*'

In [ ]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
59,Women's Store,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,1


## *Third* *cluster* *i*.*e* '*cluster* *2*'

In [ ]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,Indian Restaurant,Coffee Shop,Multicuisine Indian Restaurant,Fast Food Restaurant,Restaurant,Sandwich Place,Food Truck,2
4,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,South Indian Restaurant,Snack Place,Hookah Bar,2
12,Fast Food Restaurant,Indian Restaurant,Market,Coffee Shop,Restaurant,Sandwich Place,Bengali Restaurant,2
13,Indian Restaurant,Pizza Place,Market,BBQ Joint,Shopping Mall,Light Rail Station,Dessert Shop,2
15,Multiplex,Sandwich Place,Café,Fast Food Restaurant,Farmers Market,French Restaurant,Food Truck,2
16,Indian Restaurant,Hotel,Food & Drink Shop,Motorcycle Shop,High School,Light Rail Station,BBQ Joint,2
17,Pizza Place,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,2
23,Fast Food Restaurant,Hotel,Market,Coffee Shop,Bar,Indie Movie Theater,Flea Market,2
24,Fast Food Restaurant,Sandwich Place,Garden Center,Bakery,Gym / Fitness Center,Hookah Bar,Coffee Shop,2
28,Breakfast Spot,Chinese Restaurant,Donut Shop,Coffee Shop,Clothing Store,Grocery Store,Restaurant,2


## *fourth* *cluster* *i*.*e* '*cluster* *3*'

In [ ]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 3, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Indian Restaurant,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,3
1,Indian Restaurant,Market,Dessert Shop,Pizza Place,Asian Restaurant,Sandwich Place,Hotel,3
5,Indian Restaurant,Outdoors & Recreation,College Gym,Plaza,French Restaurant,Bistro,Snack Place,3
6,Train Station,Light Rail Station,Indian Restaurant,Hotel Bar,Farmers Market,Food Truck,Food Court,3
7,Indian Restaurant,Hotel,Lounge,Coffee Shop,Historic Site,Pub,Asian Restaurant,3
...,...,...,...,...,...,...,...,...
102,Department Store,Fast Food Restaurant,Sports Bar,Café,Moving Target,Clothing Store,Donut Shop,3
103,Metro Station,Asian Restaurant,Track,Pool,Shop & Service,Diner,Hotel,3
104,Coffee Shop,Market,Asian Restaurant,Sandwich Place,Dessert Shop,Café,Mediterranean Restaurant,3
105,Indian Restaurant,Food Truck,Coffee Shop,Restaurant,French Restaurant,Food Court,Food & Drink Shop,3


## *Fifth* *cluster* *i*.*e* '*cluster* *4*'

In [ ]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
8,Business Service,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,4


# **Analysis result:**

# Lets Finding Best location or Neighborhood for new coffee shop.

In [ ]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Botanical Garden,Boutique,Breakfast Spot,Bridal Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,...,Punjabi Restaurant,Racetrack,Record Shop,Resort,Restaurant,Road,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,Adarsh Nagar,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
1,Alaknanda,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057692,0.0,0.0,0.000000,0.0,0.038462,0.019231,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057692,0.0,0.038462,0.0,0.0,0.076923,...,0.0,0.0,0.0,0.000000,0.057692,0.0,0.0,0.057692,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019231,0.0,0.019231,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,Anand Vihar,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.0625,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.062500,...,0.0,0.0,0.0,0.000000,0.062500,0.0,0.0,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.0,0.0,0.000000,0.000000,0.0,0.111111,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.100000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.100000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.1,0.